In [ ]:
# submitted by
# Nir Vaknin

In [ ]:
# import the necessary packages
import numpy as np
import cv2
from matplotlib import pyplot as plt
import imutils
from imutils import contours

In [ ]:
# we want high contrast between digits and background, therefore we will
# read the image, convert it to grayscale and perform threshold
ocrTemplate = cv2.imread(r'C:\Users\Nir\Desktop\OCR_A_font.png')
ocrTemplate = cv2.cvtColor(ocrTemplate, cv2.COLOR_BGR2GRAY)
ocrTemplate = cv2.threshold(ocrTemplate, 10, 255, cv2.THRESH_BINARY_INV)[1]

plt.imshow(ocrTemplate)
plt.title('OCR-A font')
plt.show()

In [ ]:
# from some reason my imutils version is not up to date and i cant install the latest version
# hence, i copied imutils.grab_contours as it is
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

In [ ]:
# find the digits contours, grab them with the function that
# we build at the cell above and sort them left to right
refCnts = cv2.findContours(ocrTemplate.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
refCnts = grab_contours(refCnts)
refCnts = contours.sort_contours(refCnts, method="left-to-right")[0]
digits = {}


# loop over all the OCR-A reference contours
# compute the digit boundig box (BBox), crop, resize it
# and mapping digit name to the roi
for (i, c) in enumerate(refCnts):
    (x, y, w, h) = cv2.boundingRect(c)
    roi = ocrTemplate[y:y + h, x:x + w]
    roi = cv2.resize(roi, (115, 175))
    digits[i] = roi

# initialize a rect and square kernel's for morphological purposes
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 3))
squareKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

#plot the roi's and their numeric value
plt.figure(figsize=(18,6))
for i in range(10):
    plt.subplot(1,10,i+1)
    plt.imshow(digits[i])
    plt.title(f'numeric value={i}', fontsize = '10')
    plt.axis('off')
plt.show()

In [ ]:
################################
#begin streaming video (webcam)
################################

In [ ]:
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(0)

# update the digits prediction to the highest score prediction per digit above all frames
finalPredResult = []
finalHighestScore = []

# the number of frames we want in order to take the predictions with the higest score
framesToAnalyze = 0
scanCompleted = False

# Check if camera opened successfully
if not cap.isOpened():
    raise Exception("Could not open video device") 

# Read until video is completed
while(cap.isOpened()):
    ret, frame = cap.read()
    # cant read from some reason
    if not ret:
        break
    
    # succeed reading from the camera
    else: 
        # Display the resulting frame
        displayFrame = cv2.putText(frame, 'Verification CC box:', (130, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (220, 220, 220), 2, lineType=cv2.LINE_AA)
        if (scanCompleted):
            displayFrame = cv2.rectangle(displayFrame, (130, 110), (510, 350), (0, 255, 0), 4)
            cv2.putText(displayFrame, 'SCAN COMPLETED', (400, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), lineType=cv2.LINE_AA)
            cv2.putText(displayFrame, '4 last digits Aprox. prediction: ', (150, 380), cv2.FONT_HERSHEY_SIMPLEX, 0.7 ,(220, 220, 220), 2, lineType=cv2.LINE_AA)
            cv2.putText(displayFrame, '['+"  ".join(map(str,finalPredResult))+']', (180, 440), cv2.FONT_HERSHEY_SIMPLEX, 1.4 ,(255, 50, 50), 2, lineType=cv2.LINE_AA)
        else:
            displayFrame = cv2.rectangle(displayFrame, (130, 110), (510, 350), (0, 0, 255), 2)
            cv2.putText(displayFrame, 'Please set CC at the box above', (142, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.7 ,(220, 220, 220), 1, lineType=cv2.LINE_AA)
            cv2.putText(displayFrame, (str(framesToAnalyze) + '/15 frames deteced'), (400, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), lineType=cv2.LINE_AA)
            
       
        image = frame[110:350, 130:510]
        image = imutils.resize(image, width=300)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # disnoising image
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        gray = cv2.filter2D(gray, -1, kernel)
        
        # apply tophat (morphological operator),then compute 
        # Scharr gradient on it and scale it back to [0,255]
        tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel)
        
        # compute the Scharr gradient of the tophat image, then scale
        # the rest back into the range [0, 255]
        verticalGradient = cv2.Sobel(tophat, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)
        verticalGradient = np.absolute(verticalGradient)
        (minVal, maxVal) = (np.min(verticalGradient), np.max(verticalGradient))
        verticalGradient = (255 * ((verticalGradient - minVal) / (maxVal - minVal)))
        verticalGradient = verticalGradient.astype("uint8")
        
        # close the gaps between filtered digits using closing operation
        # and binarize image using Otsu thresholding
        verticalGradient = cv2.morphologyEx(verticalGradient, cv2.MORPH_CLOSE, rectKernel)
        thresh = cv2.threshold(verticalGradient, 0, 255,
            cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        # apply closing operation again
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, squareKernel)

        # find contours,grab them and initalize a digit locations list
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
        cnts = grab_contours(cnts)
        locs = []
            
            
        for (i, c) in enumerate(cnts):
            # extract bbox contour and calculate aspect ratio
            (x, y, w, h) = cv2.boundingRect(c)
            ar = w / float(h)
            
            # considering most credit cards using the same font size with 4 groups order,
            # we can extract the relevant contours based on the width-height and other calculations
            if ar > 2.5 and ar < 4.0:
                if (w > 40 and w < 55) and (h > 10 and h < 20):
                    # the last 4 digits contour mostly located a little bit beneath 
                    # the half height and at the most right qurter width
                    if ((x>200) and (x<250)) and ((y>100) and (y<140)):
                        # append bbox roi of digits group to locationss list
                        locs.append((x, y, w, h))
        
        # break from this frame - as we didnt find the 4 digits
        if not locs:
            cv2.imshow('Frame',displayFrame)        
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
        
        
        # loop over the 4 groupings of 4 digits (in this project only the last 4 digits)
        for (i, (gX, gY, gW, gH)) in enumerate(locs):
            # initialize the list of group digits
            groupPredResult = []
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # crop the 4 digits group roi and threshold it to highlight the digits
            group = gray[gY:gY + gH, gX:gX + gW + 2]
            group = cv2.threshold(group, 0, 255,
                cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

            # detect the contours in the 4 digits group image
            im4digits, _, _ = cv2.findContours(group.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # looking for columns of zeros in im4digits - meaning seperation between digits
            (~im4digits.any(axis=0)).any()
            seperator = np.where(~im4digits.any(axis=0))[0]
            
            digitBoxes = []
            for i in range(len(seperator)):
                # looking for seperators (columns of zeros) that not too close to each other
                if (seperator[i] - seperator[i-1] > 5):
                    digitBox = im4digits[:, seperator[i-1]:seperator[i]]
                    digitBoxes.append(digitBox)
            

        groupHighestScores = []
        # loop over the digit boxes
        for c in range(len(digitBoxes)):
            roi = cv2.resize(digitBoxes[c], (115, 175))
            # initialize a list of template matching scores
            scores = []

            # loop over the reference digit name and digit ROI
            for (digit, digitROI) in digits.items():
                # apply correlation-based template matching, take the score, and update the scores list
                # use TM_CCOEFF_NORMED (highest accuracy of all 3)
                result = cv2.matchTemplate(roi, digitROI,
                    cv2.TM_CCOEFF_NORMED)
                (_, score, _, _) = cv2.minMaxLoc(result)
                scores.append(score)

            # the classification for the digit ROI will be the reference
            # digit name with the *largest* template matching score
            digHighestScore = np.max(scores)
            groupHighestScores.append(digHighestScore)
            groupPredResult.append(np.argmax(scores))          


            # take only the groups that 4 digits (contours) has found in it
            if (len(groupPredResult) == 4):
                # enough frames to analyze
                framesToAnalyze += 1
                if (framesToAnalyze == 15):
                    scanCompleted = True

                # if we didnt detected 4 digits yet
                if not finalHighestScore:
                    finalPredResult = groupPredResult
                    finalHighestScore = groupHighestScores
                else:
                    for i in range(4):
                        if (finalHighestScore[i] < groupHighestScores[i]):
                            finalHighestScore[i] = groupHighestScores[i]
                            finalPredResult[i] = groupPredResult[i] 
                
                # plot the the digits prediction and score from the frames that have been 
                # detected for you to see validation and prediction over the frames 
                print('_____________________________________________________')
                for i in range(4):
                    print('digit #{} | prediction: {},  score: {}'.format(i+1, finalPredResult[i], finalHighestScore[i]))

        cv2.imshow('Frame',displayFrame)
        # Press Q on keyboard to  exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
#print the score of the predicted numbers (the one's with the highest score)
print('final predictions for each digit and their scores :')
for i in range(4):
    print('digit #{} | prediction: {},  score: {}'.format(i+1, finalPredResult[i], finalHighestScore[i]))